In [10]:
from loop_loss import training_loop_rl
from NER_cadec import EntityMatrixPredictor
import torch

model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_17.pth"))
model = training_loop_rl("./datasets/cadec", model=model, verbose=True, pos_weight=17, epochs=1, rl_weight=20)

Reward: -0.32758620381355286
Reward: -0.3017241358757019
Reward: -0.22413793206214905
Reward: -0.3017241358757019
Reward: -0.41379308700561523
Reward: -0.42241379618644714
Reward: -0.5
Reward: -0.25
Reward: -0.18103447556495667
Reward: -0.43103447556495667
Reward: -0.4396551847457886
Reward: -0.36206895112991333
Reward: -0.48275861144065857
Reward: -0.48275861144065857
Reward: -0.2931034564971924
Reward: -0.32758620381355286
Reward: -0.18103447556495667
Reward: -0.38793104887008667
Reward: -0.31896552443504333
Reward: -0.4913793206214905
Reward: -0.31896552443504333
Reward: -0.4396551847457886
Reward: -0.21551725268363953
Reward: -0.33620691299438477
Reward: -0.28448277711868286
Reward: -0.31896552443504333
Reward: -0.48275861144065857
Reward: -0.47413793206214905
Reward: -0.47413793206214905
Reward: -0.3017241358757019
Reward: -0.3448275923728943
Reward: -0.43103447556495667
Batch loss 0.9409356713294983
Reward: -0.36178863048553467
Reward: -0.4186992049217224
Reward: -0.4105691015720

In [11]:
import torch
torch.save(model.state_dict(), "./models/cadec/model_rlp_20.pth")

In [12]:
from NER_cadec import evaluation_loop
model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_17.pth"))
print(evaluation_loop("./datasets/cadec", model))
model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_rlp_20.pth"))
evaluation_loop("./datasets/cadec", model)

(0.20902511707109409, 0.6680272108843538, 0.31841763942931256)


(0.21922908453788875, 0.680952380952381, 0.33167660702451957)

In [ ]:
import torch

def reachability(adj_matrix):
    """
    Compute which nodes are in a cycle by checking diagonal entries of matrix powers.
    """
    n = adj_matrix.shape[0]
    reach = torch.zeros(n, dtype=torch.bool, device=adj_matrix.device)
    power = adj_matrix.clone()

    for _ in range(1, n + 1):
        reach |= torch.diag(power).bool()
        power = torch.matmul(power, adj_matrix)

    return reach

def nodes_with_arcs(adj_matrix):
    return (adj_matrix.sum(dim=0) > 0) | (adj_matrix.sum(dim=1) > 0)

def loop_reward(pred_matrix):
    """
    Compute loop reward as the fraction of nodes involved in valid loops.

    Args:
        pred_matrix (torch.Tensor): NxN binary matrix (0/1) representing entity arcs.

    Returns:
        torch.Tensor: Scalar reward between 0 and 1.
    """
    pred_binary = (pred_matrix > 0.5).int()
    upper_triangle = torch.triu(pred_binary)
    lower_positions = (torch.tril(pred_binary, diagonal=-1) == 1).nonzero(as_tuple=False)

    n = pred_binary.shape[0]
    global_loop_flags = torch.zeros(n, dtype=torch.bool, device=pred_matrix.device)

    if lower_positions.numel() == 0:
        return 1 - pred_matrix.sum() / n

    for i, j in lower_positions:
        submatrix = upper_triangle[j:i+1, j:i+1].clone()
        submatrix[i - j, 0] = 1  # Insert loop-closing arc
        local_loop_flags = reachability(submatrix)  # shape: (i-j+1,)
        global_indices = torch.arange(j, i + 1, device=pred_matrix.device)
        global_loop_flags[global_indices] |= local_loop_flags

    nodes_with_edges = nodes_with_arcs(pred_binary)
    valid_nodes = ~(global_loop_flags ^ (~nodes_with_edges))  # XNOR
    num_valid = valid_nodes.sum().float()

    return 1 - num_valid / n

A = torch.tensor([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
], dtype=torch.float32)

loop_reward(A)

tensor(0.6000)